# 2048介绍

《2048 游戏》于 2014 年由作者 Gabriele Cirulli 发布到 Github 上，后来意外走红全球，被改造成多个发行版本。由于游戏的规则简单，所以开发起来很容易，据说Gabriele Cirulli 只用了一个周末就做出了该游戏。

## 游戏规则

游戏的规则很简单，你需要控制所有方块向同一个方向运动，两个相同数字方块撞在一起之后合并成为他们的和，每次操作之后会随机生成一个2或者4，最终得到一个“2048”的方块就算胜利了。

🔥**点击 Run All Cells 即可开始游戏**，输入 W（向上）、S（向下）、A（向左）、D（向右）来控制方块的移动。

## 程序设计

### 数据结构

+ 需要一个 `row*col` 的表格来保存当前格点上的数字。
+ 需要一个变量来保存当前得分。
    > 当前得分由每次移动的得分累加得到，而每次移动的得分为所有合并数字数值的和。
+ 需要一个颜色对照表来定义每个数字对应的显示颜色。

In [ ]:
import algviz

row, col = 4, 4                                     # 定义游戏表格的行列数目。
viz = algviz.Visualizer(delay=1, wait=True)         # 待刷新的可视化对象
tab = viz.createTable(row, col, show_index=False)   # 保存数字的表格。
score = 0                                           # 保存当前得分。
color_map = {                                       # 颜色对照表。
    None: algviz.colors['white'], 2: (245, 245, 220),
    4: (250, 250, 210), 8: (240, 230, 140),
    16: (255, 215, 0), 32: (255, 165, 0),
    64: (255, 140, 0), 128: (255, 160, 122),
    256: (154, 205, 50), 512: (144, 238, 144),
    1024: (0, 128, 0), 2048: (0, 255, 0)
}

### 合法动作检测

输入动作有上、下、左、右四种，执行的操作有移动数字，或合并数字。当程序获得一个移动操作指令以后，需要执行以下步骤：

1. 检测在移动方向上有无能够直接移动的数字，如果有则返回合法，否则跳到步骤2。
2. 检测在移动方向上有无能够合并的数字，如果有则返回合法，否则返回不合法。

In [ ]:
# 检查输入动作的合法性。
def check_valid(inp):
    # 步骤1：检测有无能够直接移动的数字。
    if inp == 'A':
        for r in range(row):
            for c in range(col-1, 0, -1):
                if tab[(r, c)] is not None and tab[(r, c-1)] is None:
                    return True    # 数字和空格邻接交替出现，说明可以移动。
    elif inp == 'D':
        for r in range(row):
            for c in range(col-1):
                if tab[(r, c)] is not None and tab[(r, c+1)] is None:
                    return True
    elif inp == 'W':
        for c in range(col):
            for r in range(row-1, 0, -1):
                if tab[(r, c)] is not None and tab[(r-1, c)] is None:
                    return True
    elif inp == 'S':
        for c in range(col):
            for r in range(row-1):
                if tab[(r, c)] is not None and tab[(r+1, c)] is None:
                    return True
    # 步骤2：检测有无能够合并的数字。
    if inp == 'A' or inp == 'D':
        for r in range(row):
            for c in range(col-1):
                if tab[(r, c)] == tab[(r, c+1)]:
                    return True
    elif inp == 'W' or inp == 'S':
        for c in range(col):
            for r in range(row-1):
                if tab[(r, c)] == tab[(r+1, c)]:
                    return True
    return False

### 数字合并

根据输入的数据，沿特定方向依次检测当前数字和下一个数字是否相同。如果相同将相加后的数字复制到移动位置，并跳过下一个数字；否则直接将当前数字复制到相应位置，然后处理下一位置。

In [ ]:
def merge_nums(inp):
    global score
    if inp == 'A':
        for r in range(row):
            temp_nums = list()
            cppos = 0
            for c in range(col):
                if tab[(r, c)] is not None:
                    temp_nums.append(tab[(r, c)])
            i = 0
            while i < len(temp_nums):
                if i < len(temp_nums) - 1 and temp_nums[i] == temp_nums[i+1]:
                    tab[(r, cppos)] = temp_nums[i]*2
                    i += 1;score += temp_nums[i]*2
                else:
                    tab[(r, cppos)] = temp_nums[i]
                cppos += 1;i += 1
            for i in range(cppos, col):
                tab[(r, i)] = None
    elif inp == 'D':
        for r in range(row):
            temp_nums = list()
            cppos = col - 1
            for c in range(col-1, -1, -1):
                if tab[(r, c)] is not None:
                    temp_nums.append(tab[(r, c)])
            i = 0
            while i < len(temp_nums):
                if i < len(temp_nums) - 1 and temp_nums[i] == temp_nums[i+1]:
                    tab[(r, cppos)] = temp_nums[i]*2
                    i += 1;score += temp_nums[i]*2
                else:
                    tab[(r, cppos)] = temp_nums[i]
                cppos -= 1;i += 1
            for i in range(cppos+1):
                tab[(r, i)] = None
    elif inp == 'W':
        for c in range(col):
            temp_nums = list()
            cppos = 0
            for r in range(row):
                if tab[(r, c)] is not None:
                    temp_nums.append(tab[(r, c)])
            i = 0
            while i < len(temp_nums):
                if i < len(temp_nums) - 1 and temp_nums[i] == temp_nums[i+1]:
                    tab[(cppos, c)] = temp_nums[i]*2
                    i += 1;score += temp_nums[i]*2
                else:
                    tab[(cppos, c)] = temp_nums[i]
                cppos += 1;i += 1
            for i in range(cppos, row):
                tab[(i, c)] = None
    elif inp == 'S':
        for c in range(col):
            temp_nums = list()
            cppos = row-1
            for r in range(row-1, -1, -1):
                if tab[(r, c)] is not None:
                    temp_nums.append(tab[(r, c)])
            i = 0
            while i < len(temp_nums):
                if i < len(temp_nums) - 1 and temp_nums[i] == temp_nums[i+1]:
                    tab[(cppos, c)] = temp_nums[i]*2
                    i += 1;score += temp_nums[i]*2
                else:
                    tab[(cppos, c)] = temp_nums[i]
                cppos -= 1;i += 1
            for i in range(cppos+1):
                tab[(i, c)] = None

### 新数产生

1. 扫描数组，记录所有未填充的位置。
2. 从未填充位置中随机选择一个位置，并以0.8/0.2的概率选择数字2/4，将数字填入随机位置中。
3. 如果此时格子都满了，那么跳转到步骤4，否则返回。
4. 检测程序是否结束，即检测上、下、左、右四个方向上的动作是否合法，如果都不合法，则游戏结束。

In [ ]:
import random

def new_num():
    valid_pos = list()
    for r in range(row):
        for c in range(col):
            if tab[(r, c)] is None:
                valid_pos.append((r, c))
    pos = valid_pos[random.randint(0, len(valid_pos)-1)]
    temp_rand = random.random()
    num = 4
    if temp_rand < 0.8:
        num = 2
    tab[pos] = num
    # 当格子被放满时
    if len(valid_pos) == 1:
        if not check_valid('A') and not check_valid('D') and not check_valid('W') and not check_valid('S'):
            return False
    return True

### 主循环

1. 根据格子上的值不同为其染色，并刷新显示。
2. 读取输入并判断动作是否合法，如果合法则执行步骤3，否则再次读取输入。
3. 根据输入动作移动及合并格子中的数字，判断是否有2048存在，如果存在则停止程序，否则执行步骤4。
4. 产生新的数字并检测是否结束，如果结束则停止程序，否则执行步骤1。

In [ ]:
def markTable():
    for r in range(row):
        for c in range(col):
            tab.mark((r, c), color_map[tab[(r, c)]], hold=False)

def main():
    new_num()
    new_num()
    while True:
        for r in range(row):
            for c in range(col):
                if tab[(r, c)] == 2048:
                    viz.display()
                    return True
        markTable()
        inp = viz.display()
        if inp == 'q':
            return False
        while not check_valid(inp.upper()):
            markTable()
            inp = viz.display()
            if inp == 'q':
                return False
        merge_nums(inp.upper())
        if not new_num():
            return False
        
if main():
    print('恭喜你赢了！得分为{}'.format(score))
else:
    print('你获得了{}分，请再接再厉！'.format(score))

# 参考链接

+ [Gabriele Cirulli个人网站](https://gabrielecirulli.com/)
+ [网页版2048游戏](https://play2048.co/)
+ [宠物小精灵版2048游戏](https://filipekiss.github.io/2048/)